In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn import metrics
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
import glob

sns.set(style='white', context='notebook', rc={'figure.figsize': (14, 10)})
from tqdm import tqdm

In [3]:
models = []
model2pred = {}
model2proba = {}
y = None
mean_fpr = None
pca_comp = None
DSET_SIZE = 450

df = pd.read_csv("extracted_actor_positions_artv.csv",
                 index_col=0)\
        .drop(columns=['early_end'])\
        .sort_values(by="datetime", ascending=True)\
        .reset_index(drop=True)


pkl_list = glob.glob("ml_out2/ml_outs*.pkl")
print(len(pkl_list))
for fname in pkl_list:
    with open(fname, "rb") as f:
        ml_outs = pickle.load(f)
    config_name = f"{ml_outs['name']}${ml_outs['embedding_mode']}${ml_outs['wiki']}${ml_outs['pca']}"
    models.append(config_name)
    model2pred[config_name] = ml_outs['model_y_pred']
    model2proba[config_name] = ml_outs['model_y_proba']
    if y is None:
        y = ml_outs['y']
    else:
        assert not np.sum(y != ml_outs['y'])
    assert len(y) == len(model2pred[config_name])
    if mean_fpr is None:
        mean_fpr = ml_outs['mean_fpr']
    else:
        assert not np.sum(mean_fpr != ml_outs['mean_fpr'])
    df['traction'] = df['traction'].astype(int)
    y = y.astype(np.int64)

    assert not np.sum(y[DSET_SIZE:,] != df.iloc[DSET_SIZE:,]['traction'])

90


In [5]:
model2tprs = defaultdict(list)
model2aucs = defaultdict(list)
model2metrics = defaultdict(list)

POS_LABEL = 1

for name in tqdm(models):
    n_bootstraps = 1000

    for i in range(n_bootstraps):
        indices = np.random.randint(DSET_SIZE, len(y), len(y[DSET_SIZE:,]))
        if len(np.unique(y[indices])) < 2:
            continue

        auc = metrics.roc_auc_score(y[indices], model2proba[name][indices])
        model2aucs[name].append(auc)
        model2metrics[name].append(precision_recall_fscore_support(
            y[indices], model2pred[name][indices], average='binary', pos_label=POS_LABEL))


100%|██████████| 90/90 [02:43<00:00,  1.81s/it]


In [6]:
from pprint import pprint
model2pre = {}
model2rec = {}
model2f1 = {}
model2auc = {}
for name in model2metrics:
    pre_scores = [s[0] for s in model2metrics[name]]
    rec_scores = [s[1] for s in model2metrics[name]]
    f1_scores = [s[2] for s in model2metrics[name]]
    auc_scores = model2aucs[name]
    model2auc[name] = (np.mean(auc_scores),
                       np.quantile(auc_scores, 0.15),
                       np.quantile(auc_scores, 0.85),
                       np.std(auc_scores,ddof=1)
                       )
    model2pre[name] = (np.mean(pre_scores),
                       np.quantile(pre_scores, 0.15),
                       np.quantile(pre_scores, 0.85),
                       np.std(pre_scores, ddof=1)
                       )
    model2rec[name] = (np.mean(rec_scores),
                       np.quantile(rec_scores, 0.15),
                       np.quantile(rec_scores, 0.85),
                       np.std(rec_scores, ddof=1)
                       )
    model2f1[name] = (np.mean(f1_scores),
                      np.quantile(f1_scores, 0.15),
                      np.quantile(f1_scores, 0.85),
                      np.std(f1_scores, ddof=1)
                      )

In [7]:
d = {'model': [], 'wiki': [], 'mode': [], 'pca': [],
     'score': [], 'mean': [], 'lower': [], 'upper': [], 'sd': []}
for model in model2pre:
    d['model'].append(model.split("$")[0])
    d['wiki'].append(model.split("$")[1])
    d['mode'].append(model.split("$")[2])
    d['pca'].append(model.split("$")[3])
    d['score'].append('precision')
    d['mean'].append(model2pre[model][0])
    d['lower'].append(model2pre[model][1])
    d['upper'].append(model2pre[model][2])
    d['sd'].append(model2pre[model][3])
for model in model2rec:
    d['model'].append(model.split("$")[0])
    d['wiki'].append(model.split("$")[1])
    d['mode'].append(model.split("$")[2])
    d['pca'].append(model.split("$")[3])
    d['score'].append('recall')
    d['mean'].append(model2rec[model][0])
    d['lower'].append(model2rec[model][1])
    d['upper'].append(model2rec[model][2])
    d['sd'].append(model2rec[model][3])
for model in model2f1:
    d['model'].append(model.split("$")[0])
    d['wiki'].append(model.split("$")[1])
    d['mode'].append(model.split("$")[2])
    d['pca'].append(model.split("$")[3])
    d['score'].append('f1')
    d['mean'].append(model2f1[model][0])
    d['lower'].append(model2f1[model][1])
    d['upper'].append(model2f1[model][2])
    d['sd'].append(model2f1[model][3])
for model in model2auc:
    d['model'].append(model.split("$")[0])
    d['wiki'].append(model.split("$")[1])
    d['mode'].append(model.split("$")[2])
    d['pca'].append(model.split("$")[3])
    d['score'].append('auc')
    d['mean'].append(model2auc[model][0])
    d['lower'].append(model2auc[model][1])
    d['upper'].append(model2auc[model][2])
    d['sd'].append(model2auc[model][3])
pd.DataFrame(d)

,model,wiki,mode,pca,score,mean,lower,upper,sd
0,cast_meta,entity,late,False,precision,0.647447,0.586207,0.704950,0.057904
1,cast_text,entity,late,False,precision,0.711322,0.641509,0.777778,0.066371
2,cast,binary,late,False,precision,0.655113,0.584906,0.727273,0.070903
3,text,entity,late,False,precision,0.561512,0.500000,0.625000,0.058063
4,text_meta,entity,late,False,precision,0.653333,0.597921,0.710526,0.054657
...,...,...,...,...,...,...,...,...,...
355,prod_text_meta,binary,late,True,auc,0.711021,0.659531,0.761153,0.048121
356,prod_text,entity,late,False,auc,0.665789,0.614578,0.715007,0.048434
357,cast_prod_meta,binary,late,True,auc,0.710877,0.663269,0.758466,0.046775
358,prod_text,entity,hist,False,auc,0.631990,0.579090,0.684468,0.050036


In [8]:
pd.DataFrame(d).to_csv(f"bstrap_pre_rec_f1_062524_{POS_LABEL}_ml_artv.csv")

In [9]:
!ls -lah bstrap_pre_rec_f1_062524_1_ml_artv.csv

-rw-rw-r-- 1 jbonnell jbonnell 40K Jul  3 15:20 bstrap_pre_rec_f1_062524_1_ml_artv.csv
